In [ ]:
# === SETUP ===
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.primitives import StatevectorSampler
from qiskit.visualization import (
    plot_histogram, plot_distribution,
    plot_bloch_multivector, plot_state_city,
    plot_state_qsphere, plot_state_hinton,
    plot_state_paulivec
)
import matplotlib.pyplot as plt

sampler = StatevectorSampler()
print("✓ Setup complete")

---

## 1. Statevector vs StatevectorSampler

**Critical difference**: Statevector = NO measurements | StatevectorSampler = MUST have measurements

In [ ]:
# Statevector: Circuit WITHOUT measurements
qc_state = QuantumCircuit(2)
qc_state.h(0)
qc_state.cx(0, 1)

state = Statevector(qc_state)
print("Statevector output (exact amplitudes):")
print(state)
print(f"\nType: {type(state)}")

In [ ]:
# StatevectorSampler: Circuit WITH measurements
qc_meas = QuantumCircuit(2, 2)
qc_meas.h(0)
qc_meas.cx(0, 1)
qc_meas.measure([0, 1], [0, 1])

job = sampler.run([qc_meas], shots=1000)
result = job.result()
counts = result[0].data.c.get_counts()

print("StatevectorSampler output (simulated counts):")
print(counts)
print(f"\nType: {type(counts)}")

---

## 2. plot_histogram() - Measurement Results

In [ ]:
# Basic histogram
plot_histogram(counts)

In [ ]:
# Histogram with parameters
plot_histogram(
    counts,
    title='Bell State Measurements',
    color='steelblue',
    figsize=(8, 4),
    legend=['Bell State']
)

In [ ]:
# Compare multiple experiments
qc_h = QuantumCircuit(1, 1)
qc_h.h(0)
qc_h.measure(0, 0)

qc_x = QuantumCircuit(1, 1)
qc_x.x(0)
qc_x.measure(0, 0)

job_both = sampler.run([qc_h, qc_x], shots=1000)
counts_h = job_both.result()[0].data.c.get_counts()
counts_x = job_both.result()[1].data.c.get_counts()

plot_histogram([counts_h, counts_x], legend=['H gate', 'X gate'])

---

## 3. plot_bloch_multivector() - Geometric View

In [ ]:
# Single qubit states
qc0 = QuantumCircuit(1)  # |0>
qc1 = QuantumCircuit(1); qc1.x(0)  # |1>
qc_plus = QuantumCircuit(1); qc_plus.h(0)  # |+>

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
for ax, qc, label in zip(axes, [qc0, qc1, qc_plus], ['|0>', '|1>', '|+>']):
    plot_bloch_multivector(Statevector(qc), ax=ax)
    ax.set_title(label)
plt.tight_layout()

In [ ]:
# Multi-qubit: DETECTS ENTANGLEMENT!
# Entangled qubits show arrows at CENTER (origin)
qc_bell = QuantumCircuit(2)
qc_bell.h(0)
qc_bell.cx(0, 1)

print("Entangled Bell state - arrows at CENTER:")
plot_bloch_multivector(Statevector(qc_bell))

---

## 4. plot_state_city() - 3D Amplitude Bars

In [ ]:
# Shows real and imaginary parts of amplitudes
state_bell = Statevector(qc_bell)
plot_state_city(state_bell, title='Bell State Amplitudes')

In [ ]:
# With phase - creates imaginary component
qc_phase = QuantumCircuit(1)
qc_phase.h(0)
qc_phase.s(0)  # Adds phase to |1>

plot_state_city(Statevector(qc_phase), title='State with Phase (S gate)')

---

## 5. plot_state_qsphere() - Probability + Phase

In [ ]:
# Shows probability (size) and phase (color)
plot_state_qsphere(state_bell)

In [ ]:
# Different phase states
qc_t = QuantumCircuit(1)
qc_t.h(0)
qc_t.t(0)  # pi/4 phase

plot_state_qsphere(Statevector(qc_t))
plt.title('T gate phase on |1>')

---

## 6. plot_state_hinton() - Density Matrix

In [ ]:
# Square sizes show magnitude, colors show sign
plot_state_hinton(state_bell, title='Bell State Density Matrix')

---

## 7. plot_state_paulivec() - Pauli Decomposition

In [ ]:
# Expectation values of Pauli operators
plot_state_paulivec(state_bell, title='Bell State Pauli Decomposition')

---

## TRAP DEMONSTRATIONS

In [ ]:
# TRAP 1: Statevector with measurements - ERROR!
qc_with_meas = QuantumCircuit(1, 1)
qc_with_meas.h(0)
qc_with_meas.measure(0, 0)

try:
    state_bad = Statevector(qc_with_meas)
except Exception as e:
    print(f"ERROR: {type(e).__name__}")
    print(f"  Statevector cannot have measurements!")
    print(f"  FIX: Remove measurements or use StatevectorSampler")

In [ ]:
# TRAP 2: State visualization with counts - TypeError!
try:
    plot_state_city(counts)  # counts is a dict, not Statevector!
except Exception as e:
    print(f"ERROR: {type(e).__name__}")
    print(f"  plot_state_city expects Statevector, not dict!")
    print(f"  FIX: Use plot_histogram(counts) for measurement results")

In [ ]:
# TRAP 3: Histogram with Statevector - TypeError!
try:
    plot_histogram(state_bell)  # state is Statevector, not counts!
except Exception as e:
    print(f"ERROR: {type(e).__name__}")
    print(f"  plot_histogram expects counts dict, not Statevector!")
    print(f"  FIX: Use plot_state_city(state) for Statevector")

In [ ]:
# TRAP 4: Sampler without measurements - No counts!
qc_no_meas = QuantumCircuit(1)
qc_no_meas.h(0)

job_bad = sampler.run([qc_no_meas], shots=1000)
result_bad = job_bad.result()

print("Result data keys:", dir(result_bad[0].data))
print("\nNo classical register = no .get_counts() available!")
print("FIX: Add measurements to circuit before running Sampler")

---

## VERIFICATION TESTS

In [ ]:
# Test 1: Complete state visualization workflow
def test_state_viz():
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    
    state = Statevector(qc)
    
    # All state visualizations should work
    assert plot_bloch_multivector(state) is not None
    assert plot_state_city(state) is not None
    assert plot_state_qsphere(state) is not None
    assert plot_state_hinton(state) is not None
    assert plot_state_paulivec(state) is not None
    plt.close('all')
    return True

print("State visualization test:", test_state_viz())

In [ ]:
# Test 2: Complete histogram workflow
def test_histogram():
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])
    
    job = sampler.run([qc], shots=1000)
    counts = job.result()[0].data.c.get_counts()
    
    assert plot_histogram(counts) is not None
    plt.close('all')
    return True

print("Histogram test:", test_histogram())

In [ ]:
# Test 3: GHZ state verification
def test_ghz():
    qc = QuantumCircuit(3)
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(1, 2)
    
    state = Statevector(qc)
    probs = state.probabilities()
    
    # GHZ: only |000> and |111> should have probability
    assert abs(probs[0] - 0.5) < 0.01  # |000>
    assert abs(probs[7] - 0.5) < 0.01  # |111>
    return True

print("GHZ state test:", test_ghz())

---

## Quick Decision Guide

```
What data do you have?
+-- Counts dict (from measurements)
|   +-- plot_histogram(counts)
|
+-- Statevector (no measurements)
    +-- Single qubit geometric view -> plot_bloch_multivector()
    +-- See exact amplitudes -> plot_state_city()
    +-- See probability + phase -> plot_state_qsphere()
    +-- See density matrix -> plot_state_hinton()
    +-- Pauli decomposition -> plot_state_paulivec()
```

---

## `plot_distribution()` - Normalized Probabilities

### Signature
```python
plot_distribution(data, figsize=None, color=None, legend=None, title=None, **kwargs)
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `data` | `dict` or `list[dict]` | Yes | - | Counts dictionary from measurement |
| `figsize` | `tuple` | No | `None` | Figure size (width, height) |
| `color` | `str` or `list` | No | `None` | Bar color(s) |
| `legend` | `list[str]` | No | `None` | Labels for multiple distributions |
| `title` | `str` | No | `None` | Plot title |

### Returns
`matplotlib.figure.Figure` - The distribution plot

### Key Difference from `plot_histogram()`

| Aspect | `plot_histogram()` | `plot_distribution()` |
|--------|-------------------|----------------------|
| **Y-axis** | Raw counts (integers) | Probabilities (0.0 to 1.0) |
| **Sum of bars** | Total shots | 1.0 |
| **Use case** | Compare experiments | Theoretical analysis |
| **Example** | `{'00': 500, '11': 500}` → bars at 500 | `{'00': 500, '11': 500}` → bars at 0.5 |

In [ ]:
# ============================================================
# plot_distribution() - BASIC USAGE
# ============================================================

# Same counts, different visualizations
counts_bell = {'00': 520, '11': 480}

print("COMPARISON: plot_histogram vs plot_distribution")
print("="*55)

import matplotlib.pyplot as plt
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Left: Histogram (raw counts)
from qiskit.visualization import plot_histogram, plot_distribution
plot_histogram(counts_bell, ax=axes[0], title='plot_histogram (Raw Counts)')
axes[0].set_ylabel('Counts')

# Right: Distribution (normalized probabilities)
plot_distribution(counts_bell, ax=axes[1], title='plot_distribution (Probabilities)')
axes[1].set_ylabel('Probability')

plt.tight_layout()
plt.show()

print("\n📊 Key Insight:")
print("  histogram: bars sum to 1000 (total shots)")
print("  distribution: bars sum to 1.0 (normalized)")
print("✅ Use plot_distribution when comparing to theoretical predictions")

In [ ]:
# ============================================================
# ⚠️ TRAP: plot_distribution also needs counts dict!
# ============================================================

print("⚠️ TRAP: Same input requirement as plot_histogram")
print("="*55)

# ❌ WRONG - Can't use Statevector
try:
    plot_distribution(state_bell)  # state_bell is Statevector, not counts!
except Exception as e:
    print(f"❌ plot_distribution(Statevector) → {type(e).__name__}")
    print(f"   Error: {str(e)[:50]}...")

# ✅ CORRECT - Use counts dictionary
print("\n✅ CORRECT: plot_distribution(counts_dict)")
print("   Works with: {'00': 520, '11': 480}")

# Verification
def verify_distribution():
    """Verify plot_distribution works with counts."""
    counts = {'00': 500, '11': 500}
    fig = plot_distribution(counts)
    assert fig is not None
    plt.close(fig)
    return True

print(f"\n✅ plot_distribution verification: {verify_distribution()}")

---

## 1. Statevector vs StatevectorSampler

**Critical difference**: Statevector = NO measurements | StatevectorSampler = MUST have measurements

In [ ]:
# Statevector: Circuit WITHOUT measurements
qc_state = QuantumCircuit(2)
qc_state.h(0)
qc_state.cx(0, 1)

state = Statevector(qc_state)
print("Statevector output (exact amplitudes):")
print(state)
print(f"\nType: {type(state)}")

In [ ]:
# StatevectorSampler: Circuit WITH measurements
qc_meas = QuantumCircuit(2, 2)
qc_meas.h(0)
qc_meas.cx(0, 1)
qc_meas.measure([0, 1], [0, 1])

job = sampler.run([qc_meas], shots=1000)
result = job.result()
counts = result[0].data.c.get_counts()

print("StatevectorSampler output (simulated counts):")
print(counts)
print(f"\nType: {type(counts)}")

---

## 2. plot_histogram() - Measurement Results

In [ ]:
# Basic histogram
plot_histogram(counts)

In [ ]:
# Histogram with parameters
plot_histogram(
    counts,
    title='Bell State Measurements',
    color='steelblue',
    figsize=(8, 4),
    legend=['Bell State']
)

In [ ]:
# Compare multiple experiments
qc_h = QuantumCircuit(1, 1)
qc_h.h(0)
qc_h.measure(0, 0)

qc_x = QuantumCircuit(1, 1)
qc_x.x(0)
qc_x.measure(0, 0)

job_both = sampler.run([qc_h, qc_x], shots=1000)
counts_h = job_both.result()[0].data.c.get_counts()
counts_x = job_both.result()[1].data.c.get_counts()

plot_histogram([counts_h, counts_x], legend=['H gate', 'X gate'])

---

## 3. plot_bloch_multivector() - Geometric View

In [ ]:
# Single qubit states
qc0 = QuantumCircuit(1)  # |0⟩
qc1 = QuantumCircuit(1); qc1.x(0)  # |1⟩
qc_plus = QuantumCircuit(1); qc_plus.h(0)  # |+⟩

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
for ax, qc, label in zip(axes, [qc0, qc1, qc_plus], ['|0⟩', '|1⟩', '|+⟩']):
    plot_bloch_multivector(Statevector(qc), ax=ax)
    ax.set_title(label)
plt.tight_layout()

In [ ]:
# Multi-qubit: DETECTS ENTANGLEMENT!
# Entangled qubits show arrows at CENTER (origin)
qc_bell = QuantumCircuit(2)
qc_bell.h(0)
qc_bell.cx(0, 1)

print("Entangled Bell state - arrows at CENTER:")
plot_bloch_multivector(Statevector(qc_bell))

---

## 4. plot_state_city() - 3D Amplitude Bars

In [ ]:
# Shows real and imaginary parts of amplitudes
state_bell = Statevector(qc_bell)
plot_state_city(state_bell, title='Bell State Amplitudes')

In [ ]:
# With phase - creates imaginary component
qc_phase = QuantumCircuit(1)
qc_phase.h(0)
qc_phase.s(0)  # Adds phase to |1⟩

plot_state_city(Statevector(qc_phase), title='State with Phase (S gate)')

---

## 5. plot_state_qsphere() - Probability + Phase

In [ ]:
# Shows probability (size) and phase (color)
plot_state_qsphere(state_bell)

In [ ]:
# Different phase states
qc_t = QuantumCircuit(1)
qc_t.h(0)
qc_t.t(0)  # π/4 phase

plot_state_qsphere(Statevector(qc_t))
plt.title('T gate phase on |1⟩')

---

## 6. plot_state_hinton() - Density Matrix

In [ ]:
# Square sizes show magnitude, colors show sign
plot_state_hinton(state_bell, title='Bell State Density Matrix')

---

## 7. plot_state_paulivec() - Pauli Decomposition

In [ ]:
# Expectation values of Pauli operators
plot_state_paulivec(state_bell, title='Bell State Pauli Decomposition')

---

## ⚠️ TRAP DEMONSTRATIONS

In [ ]:
# TRAP 1: Statevector with measurements - ERROR!
qc_with_meas = QuantumCircuit(1, 1)
qc_with_meas.h(0)
qc_with_meas.measure(0, 0)

try:
    state_bad = Statevector(qc_with_meas)
except Exception as e:
    print(f"✗ ERROR: {type(e).__name__}")
    print(f"  Statevector cannot have measurements!")
    print(f"  ✓ FIX: Remove measurements or use StatevectorSampler")

In [ ]:
# TRAP 2: State visualization with counts - TypeError!
try:
    plot_state_city(counts)  # counts is a dict, not Statevector!
except Exception as e:
    print(f"✗ ERROR: {type(e).__name__}")
    print(f"  plot_state_city expects Statevector, not dict!")
    print(f"  ✓ FIX: Use plot_histogram(counts) for measurement results")

In [ ]:
# TRAP 3: Histogram with Statevector - TypeError!
try:
    plot_histogram(state_bell)  # state is Statevector, not counts!
except Exception as e:
    print(f"✗ ERROR: {type(e).__name__}")
    print(f"  plot_histogram expects counts dict, not Statevector!")
    print(f"  ✓ FIX: Use plot_state_city(state) for Statevector")

In [ ]:
# TRAP 4: Sampler without measurements - No counts!
qc_no_meas = QuantumCircuit(1)
qc_no_meas.h(0)

job_bad = sampler.run([qc_no_meas], shots=1000)
result_bad = job_bad.result()

print("Result data keys:", dir(result_bad[0].data))
print("\n✗ No classical register = no .get_counts() available!")
print("✓ FIX: Add measurements to circuit before running Sampler")

---

## ✅ VERIFICATION TESTS

In [ ]:
# Test 1: Complete state visualization workflow
def test_state_viz():
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    
    state = Statevector(qc)
    
    # All state visualizations should work
    assert plot_bloch_multivector(state) is not None
    assert plot_state_city(state) is not None
    assert plot_state_qsphere(state) is not None
    assert plot_state_hinton(state) is not None
    assert plot_state_paulivec(state) is not None
    plt.close('all')
    return True

print("✓ State visualization test:", test_state_viz())

In [ ]:
# Test 2: Complete histogram workflow
def test_histogram():
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])
    
    job = sampler.run([qc], shots=1000)
    counts = job.result()[0].data.c.get_counts()
    
    assert plot_histogram(counts) is not None
    plt.close('all')
    return True

print("✓ Histogram test:", test_histogram())

In [ ]:
# Test 3: GHZ state verification
def test_ghz():
    qc = QuantumCircuit(3)
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(1, 2)
    
    state = Statevector(qc)
    probs = state.probabilities()
    
    # GHZ: only |000⟩ and |111⟩ should have probability
    assert abs(probs[0] - 0.5) < 0.01  # |000⟩
    assert abs(probs[7] - 0.5) < 0.01  # |111⟩
    return True

print("✓ GHZ state test:", test_ghz())

---

## 📋 Quick Decision Guide

```
What data do you have?
├── Counts dict (from measurements)
│   └── plot_histogram(counts)
│
└── Statevector (no measurements)
    ├── Single qubit geometric view → plot_bloch_multivector()
    ├── See exact amplitudes → plot_state_city()
    ├── See probability + phase → plot_state_qsphere()
    ├── See density matrix → plot_state_hinton()
    └── Pauli decomposition → plot_state_paulivec()
```

---

## 🏆 CODE CHALLENGES

Test your state visualization skills with these hands-on challenges.

In [ ]:
# ============================================================
# CHALLENGE 1: Visualize All Six Cardinal States
# ============================================================
# Create circuits for |0⟩, |1⟩, |+⟩, |-⟩, |+i⟩, |-i⟩
# and visualize them all on Bloch spheres using subplots

# YOUR CODE HERE:
states = {
    '|0⟩': QuantumCircuit(1),  # Already |0⟩
    '|1⟩': None,  # Apply X gate
    '|+⟩': None,  # Apply H gate
    '|-⟩': None,  # Apply X then H
    '|+i⟩': None,  # Apply H then S
    '|-i⟩': None,  # Apply H then Sdg
}

# Fill in the circuits above, then:
# fig, axes = plt.subplots(2, 3, figsize=(15, 10))
# for ax, (label, qc) in zip(axes.flat, states.items()):
#     if qc:
#         plot_bloch_multivector(Statevector(qc), ax=ax)
#         ax.set_title(label)

# Verification
def verify_challenge_1():
    """Verify all six cardinal states."""
    test_states = {
        '|0⟩': [1, 0],
        '|1⟩': [0, 1],
    }
    qc0 = QuantumCircuit(1)
    qc1 = QuantumCircuit(1); qc1.x(0)
    
    sv0 = Statevector(qc0)
    sv1 = Statevector(qc1)
    
    assert abs(sv0.data[0] - 1) < 0.01, "|0⟩ wrong"
    assert abs(sv1.data[1] - 1) < 0.01, "|1⟩ wrong"
    print("✅ Challenge 1 PASSED!")

# Uncomment to verify:
# verify_challenge_1()

In [ ]:
# ============================================================
# CHALLENGE 2: Superposition vs Entanglement Histogram
# ============================================================
# Create TWO circuits:
# 1. Superposition: H on both qubits (independent)
# 2. Entanglement: H on qubit 0, then CNOT (correlated)
# Run both and compare histograms - explain the difference

# YOUR CODE HERE:
qc_super = QuantumCircuit(2, 2)
# Add H gates for superposition...

qc_entang = QuantumCircuit(2, 2)
# Add H + CNOT for entanglement...

# Verification
def verify_challenge_2(qc_s, qc_e):
    """Verify superposition vs entanglement."""
    # Run both
    job = sampler.run([qc_s, qc_e], shots=1000)
    counts_s = job.result()[0].data.c.get_counts()
    counts_e = job.result()[1].data.c.get_counts()
    
    # Superposition: should have all 4 outcomes
    assert len(counts_s) == 4, "Superposition should have 4 outcomes"
    
    # Entanglement: should only have 00 and 11
    assert len(counts_e) == 2, "Entanglement should have 2 outcomes"
    assert '00' in counts_e and '11' in counts_e, "Should be 00 and 11 only"
    
    print("✅ Challenge 2 PASSED!")

# Uncomment to verify (after adding measurements):
# verify_challenge_2(qc_super, qc_entang)

In [ ]:
# ============================================================
# CHALLENGE 3: State Evolution Visualization
# ============================================================
# Create a Bell state step by step and visualize state at each step:
# 1. Initial |00⟩ state
# 2. After H on qubit 0
# 3. After CNOT(0, 1)
# Use plot_state_city for each step to see amplitude changes

# YOUR CODE HERE:
def visualize_bell_evolution():
    """Visualize Bell state creation step by step."""
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Step 1: Initial state
    qc1 = QuantumCircuit(2)
    # visualize...
    
    # Step 2: After H
    qc2 = QuantumCircuit(2)
    qc2.h(0)
    # visualize...
    
    # Step 3: After CNOT (Bell state)
    qc3 = QuantumCircuit(2)
    qc3.h(0)
    qc3.cx(0, 1)
    # visualize...
    
    plt.tight_layout()
    return qc1, qc2, qc3

# Verification
def verify_challenge_3():
    """Verify Bell state evolution."""
    qc_final = QuantumCircuit(2)
    qc_final.h(0)
    qc_final.cx(0, 1)
    
    state = Statevector(qc_final)
    probs = state.probabilities()
    
    # Should be ~0.5 for |00⟩ and |11⟩
    assert abs(probs[0] - 0.5) < 0.01
    assert abs(probs[3] - 0.5) < 0.01
    print("✅ Challenge 3 PASSED!")

# Uncomment to verify:
# verify_challenge_3()